In [66]:
import sqlite3
from sqlite3 import Error

**Preprocessing**

*Создание и подключение БД*

In [67]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [68]:
mydb_con = create_connection('my_db')

Connection to SQLite DB successful


In [69]:
s_a_p_db = mydb_con.cursor()

In [70]:
def add_data (connection, data):
    cursor = connection.cursor()
    try:
        cursor.execute(data)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [94]:
sales_table = "CREATE TABLE IF NOT EXISTS sales_data (calday DATA, region TEXT, prod_s_id INT, rev REAL, qty INT)"

In [95]:
add_data(mydb_con, sales_table)

Query executed successfully


In [96]:
prod_table = "CREATE TABLE IF NOT EXISTS prod_data (prod_id INT, prod_name TEXT, cat_name TEXT)"

In [97]:
add_data(mydb_con, prod_table)

Query executed successfully


In [92]:
s_a_p_db.execute('DROP TABLE sales_data')

*Наполнение БД*

In [136]:
add_s = """INSERT INTO sales_data (calday, region, prod_s_id, rev, qty) VALUES 
('2017-07-09', 'Амурская обл.', 200002, 100, 1), 
('2016-01-13', 'Санкт-Петербург', 38877, 10000, 5), 
('2018-06-20', 'San-Francisco', 10001, 20000000, 1),
('2020-05-21', 'Москва', 666, 666666, 6)"""

In [135]:
add_data(mydb_con, add_s)   

Query executed successfully


In [101]:
add_d = """INSERT INTO prod_data (prod_id, prod_name, cat_name) VALUES 
(2000002, "Баралгин", "Таблетки от кашля"), 
(10001, "Tesla Model X Green", "Автомобили"), 
(19992, "Панассоник Meracoc-5000", "Пылесосы"), 
(666, "Лучший в мире пылесос", "Пылесосы")"""

In [132]:
add_data(mydb_con, add_d)   

Query executed successfully


**Задание 1**

Сколько продали пылесосов
a. штук,
b. на какую сумму,
c. какая средняя цена.

In [142]:
vacs_data = s_a_p_db.execute("""SELECT rev, qty
FROM sales_data as s
INNER JOIN (SELECT * FROM prod_data WHERE cat_name = 'Пылесосы') as p  
    ON p.prod_id == s.prod_s_id
    """).fetchall()

In [144]:
vacs_data

[(666666.0, 6)]

In [143]:
total_items = 0
total_sum = 0
for item in vacs_data:
    total_sum = total_sum + item[0]
    total_items = total_items + item[1]
mean_price = total_sum / total_items
print("Всего пылесосов было продано: " + str(total_items))
print("Общая выручка с пылесосов: " + str(total_sum))
print("Средняя цена пылесоса: " + str(mean_price))

Всего пылесосов было продано: 6
Общая выручка с пылесосов: 666666.0
Средняя цена пылесоса: 111111.0


**Задача 2**

Топ-5 регионов по продажам холодильников за май 2018.

In [182]:
#ещё один пылесос, который может входить в топ
add_s = """INSERT INTO sales_data (calday, region, prod_s_id, rev, qty) VALUES 
('2018-05-21', 'Сызрань', 121212, 1212, 12)"""
add_d = """INSERT INTO prod_data (prod_id, prod_name, cat_name) VALUES 
 (121212, "Отличный пылесос", "Пылесосы")"""
add_data(mydb_con, add_s)   
add_data(mydb_con, add_в)   

In [185]:
all_top = s_a_p_db.execute("""SELECT calday, region, sum(qty) as sm
FROM sales_data  as s
INNER JOIN (SELECT * FROM prod_data WHERE cat_name = 'Пылесосы') as p  
    ON p.prod_id == s.prod_s_id
WHERE calday LIKE '%2018-05%'
GROUP BY region
ORDER BY sm LIMIT 5""").fetchall()
all_top

[('2018-05-21', 'Сызрань', 12)]

In [186]:
for item in all_top:
    print (item[1])

Сызрань


**Задание 2.1**

Может, я как-то неправильно поняла задание?

In [1]:
my_mas = [1, 4, 6, 3, 4, 6, 2]

In [2]:
sorted(my_mas)

[1, 2, 3, 4, 4, 6, 6]

In [3]:
max(my_mas)

6

In [4]:
min(my_mas)

1

**Задание 2.2**

In [22]:
import pandas as pd
import numpy as np
import os

In [16]:
a = []
for df in pd.read_csv('lil_data.csv',sep=',', header = None, chunksize=1):
    a.append(df)

In [18]:
inp = [{'f1':'a', 'f2':1}, {'f1':'a','f2':2}, {'f1':'b','f2':3}, {'f1':'c','f2':4}]
my_df = pd.DataFrame(inp)

In [19]:
my_df

,f1,f2
0,a,1
1,a,2
2,b,3
3,c,4


In [36]:
for index, row in my_df.iterrows():
    name_file = row['f1'] + '_data.txt'
    if not os.path.exists(name_file):
        f=open(name_file, "w", encoding="UTF-8")
    else:
        f=open(name_file, "a", encoding="UTF-8")
    f.write(row['f1'])
    f.write('\t')
    f.write(str(row['f2']))
    f.write('\n')
    f.close()